In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
a = pd.read_csv("../기상청공모전/data/derivate_data.csv")

In [3]:
a1 = a.iloc[:,1:]

In [4]:
a1 = a1.drop('hdg',axis = 1)

In [5]:
a1 = a1[a1["cog"] != -99.9]

In [6]:
a1

,anchor_drag,num,latitude,longitude,sog,cog,d_lati,d_long,d_d_lati,d_d_long
0,0,1001,35.461472,129.439878,0.4,9.6,0.000312,0.000333,0.000342,0.000342
1,0,1001,35.461688,129.439792,0.2,328.0,0.000216,-0.000086,-0.000096,-0.000096
2,0,1001,35.461708,129.439733,0.2,260.1,0.000020,-0.000059,-0.000196,-0.000196
3,0,1001,35.461720,129.439708,0.1,262.8,0.000012,-0.000025,-0.000008,-0.000008
4,0,1001,35.461733,129.439717,0.3,277.6,0.000013,0.000009,0.000001,0.000001
...,...,...,...,...,...,...,...,...,...,...
681435,1,4087,35.053083,129.061333,1.3,26.0,0.000033,0.000016,-0.000017,-0.000017
681436,1,4087,35.053083,129.061333,1.3,29.0,0.000000,0.000000,-0.000033,-0.000033
681437,1,4087,35.053117,129.061400,1.2,48.0,0.000034,0.000067,0.000034,0.000034
681438,1,4087,35.053117,129.061417,1.2,52.0,0.000000,0.000017,-0.000034,-0.000034


In [7]:
# ANN
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

X = a1.iloc[:, 2:]  # 세번째 열부터 마지막 열까지 입력값
y = a1.iloc[:, 0]  # 첫번째 열이 정답열


# 학습 데이터와 테스트 데이터로 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the ANN model architecture
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, verbose=1)

# Predict on the test set
y_pred_prob = model.predict(X_test)
y_pred_binary = [1 if y >= 0.5 else 0 for y in y_pred_prob]

# Evaluate accuracy
accuracy = accuracy_score(y_test, y_pred_binary)
print("Accuracy:", accuracy)

Epoch 1/10
16903/16903 [==============================] - 29s 2ms/step - loss: 0.0807 - accuracy: 0.9816
Epoch 2/10
16903/16903 [==============================] - 27s 2ms/step - loss: 0.0662 - accuracy: 0.9830
Epoch 3/10
16903/16903 [==============================] - 26s 2ms/step - loss: 0.0654 - accuracy: 0.9830
Epoch 4/10
16903/16903 [==============================] - 27s 2ms/step - loss: 0.0652 - accuracy: 0.9831
Epoch 5/10
16903/16903 [==============================] - 30s 2ms/step - loss: 0.0648 - accuracy: 0.9830
Epoch 6/10
16903/16903 [==============================] - 30s 2ms/step - loss: 0.0648 - accuracy: 0.9831
Epoch 7/10
16903/16903 [==============================] - 29s 2ms/step - loss: 0.0646 - accuracy: 0.9831
Epoch 8/10
16903/16903 [==============================] - 27s 2ms/step - loss: 0.0645 - accuracy: 0.9831
Epoch 9/10
16903/16903 [==============================] - 29s 2ms/step - loss: 0.0645 - accuracy: 0.9832
Epoch 10/10
4226/4226 [==============================] 

In [10]:
from sklearn.metrics import f1_score
print(f1_score(y_test, y_pred_binary))

0.4692737430167597


In [11]:
from sklearn.metrics import confusion_matrix

# 혼동 행렬 계산
cm = confusion_matrix(y_test, y_pred_binary)

# CSI 계산
tp = cm[1, 1]  # True Positive
tn = cm[0, 0]  # True Negative
fp = cm[0, 1]  # False Positive
fn = cm[1, 0]  # False Negative

csi = tp / (tp + fn + fp)
print("CSI 점수:", csi)

CSI 점수: 0.30656934306569344
